# Unified Plotting for NuMu Selection

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pi0scaling = {"val": 0, "suffix": "", "title": "No pi0 Scaling"} #no scaling

GENIEtune = "updated GENIE tune"

In [ ]:
from matplotlib import pyplot as plt

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
plt.rcParams.update(params)

from load_data_run123 import *
from unblinding_far_sideband import *

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
import scipy.stats

In [ ]:
which_sideband = ['numu',"numu"]

In [ ]:
samples = load_data_run123(which_sideband=which_sideband[0],
                           return_plotter=False,
                           pi0scaling=pi0scaling["val"],
                           loadpi0variables=False,
                           loadtruthfilters=False,
                           loadshowervariables=False,
                           loadnumuntuples=True,
                           loadnumuvariables=True,
                           loadnumucrtonly=True, # use CRT
                           loadfakedata=0,
                           USEBDT=False,
                           loadsystematics=True,
                           loadrecoveryvars=False)

In [ ]:
import plotter
import unblinding_far_sideband
import importlib
import load_data_run123
importlib.reload(plotter)
importlib.reload(unblinding_far_sideband)
importlib.reload(load_data_run123)

In [ ]:
plotter_runbyrun = {}
for run in [1, 2, 3, 123, 12, 30]:
    this_weights, this_pot = get_weights(run,dataset=which_sideband[1])
    if (run == 30):
        print (this_weights)
    plotter_runbyrun[run] = plotter.Plotter(samples, this_weights, pot=this_pot)

In [ ]:
DETSYS = {'mc':True,'nue':False,'ccpi0':False,'ncpi0':False,'dirt':False,\
          'ncnopi':False,'nccpi':False,'ncpi0':False,'ccpi0':False}

# Fig. 4 $\nu_{\mu}$ CC Ereco Spectrum

# Single Variable Plotter

In [ ]:
from unblinding_far_sideband import *

sdb_key = 'None' #'None' #'HiEZP' #'None' #'None'
pre_key = 'NUMU' #'HiEZP' #'NP'
sel_key = 'NUMUCRT' #'NPBDT'

run = 30

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]
sel =  selection_categories[sel_key]

DETSYSPATH = ls.ntuple_path+'/detsys/{}_{}_{}/'.format('None', preselection['dir'], sel['dir'])
print (DETSYSPATH)
QUERY = ""
ctr = 0
for query in [sideband['query'],preselection['query'],sel['query']]:
    ctr += 1
    if query is None: continue
    if QUERY: 
        if (ctr == 2):
            QUERY += ' and '
        if (ctr == 3):
            QUERY += ' and '# ~('
    QUERY += query
    #if (ctr == 3): QUERY += ')'
    if run in [1, 2, 3, 12]:
        QUERY += f' and run{run}==True'

TITLE = ""
for text in [f'Run {run}',' - ',sideband['title'],' - ',preselection['title'],'\n',sel['title'],' - ',pi0scaling["title"] + ' and ' + GENIEtune]:
    if text is None: continue
    TITLE += text
TITLE = r'$\nu_{\mu}$ selection'

#VARIABLE, BINS, RANGE, XTIT = 'muon_energy',14,(0.15,1.55),"muon candidate reconstructed energy [GeV]"
#VARIABLE, BINS, RANGE, XTIT = 'muon_theta',20,(-1,1),r"muon $\cos\theta$"
#VARIABLE, BINS, RANGE, XTIT = 'proton_range_energy',30,(0,0.3),"proton kinetic energy [GeV]" # proton_KE
VARIABLE, BINS, RANGE, XTIT = 'neutrino_energy',14,(0.15,1.55),"Reconstructed Neutrino Energy [GeV]"
#VARIABLE, BINS, RANGE, XTIT = 'neutrino_energy',24,(0.15,2.55),"Reconstructed Neutrino Energy [GeV]"

KIND = 'paper_category_numu'

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARIABLE,   
    query=QUERY,
    figtitle="MicroBooNE",
    #labeldecimals=0,
    predictedevents=False,
    kind =KIND,
    #kind="interaction",
    draw_sys=True,
    detsysdict=DETSYS,
    DETSYSPATH=DETSYSPATH,
    #genieweight="weightSpline",
    COVMATRIX="",
    ratio=False,
    fullcov=False,
    stacksort=3,
    title=XTIT,
    bins=BINS,
    range=RANGE,
    chisq=False,
    #asymErrs=True
)[0:3]


ax1.set_title(TITLE, loc='left')

#ax1.set_yscale('log')
#ax1.set_ylim(0, ax1.get_ylim()[1]*1.35)
ax1.set_ylim(0,3000)
#ax2.set_ylim(0.5,1.5)

plt.tight_layout()
fig.show()
SELNAME = '_{}_{}_{}_'.format(sideband['dir'], preselection['dir'], sel['dir'])
save_path = '/Users/davidc-local/Neutrinos/NUE/PRD/' + SELNAME + VARIABLE
#fig.savefig(save_path + '.pdf', dpi=250)    